In [2]:
import pandas as pd
import numpy as np
import requests
import json



In [77]:
# read your StreamingHistory files into pandas dataframes
df_stream = pd.read_json('StreamingHistory0.json')




In [78]:
# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()


,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-09-13 11:36,Burna Boy,Level Up (Twice As Tall) [feat. Youssou N'Dour],80822,Burna Boy:Level Up (Twice As Tall) [feat. Yous...
1,2020-09-30 11:07,Burna Boy,Level Up (Twice As Tall) [feat. Youssou N'Dour],2990,Burna Boy:Level Up (Twice As Tall) [feat. Yous...
2,2020-09-30 11:10,Burna Boy,Alarm Clock,132626,Burna Boy:Alarm Clock
3,2020-09-30 11:13,Burna Boy,Way Too Big,182743,Burna Boy:Way Too Big
4,2020-09-30 11:16,Tatiana Manaois,Got You Upset,0,Tatiana Manaois:Got You Upset


In [79]:
# read your edited Library json file into a pandas dataframe
df_library = pd.read_json('YourLibrary1.json')
# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()





,artist,album,track,uri,UniqueID,track_uri
0,Olly Murs,You Know I Know (Deluxe),You Know I Know (feat. Shaggy),spotify:track:7yATeGiKEOcljVe1yet9zx,Olly Murs:You Know I Know (feat. Shaggy),7yATeGiKEOcljVe1yet9zx
1,Olly Murs,You Know I Know (Deluxe),You Don't Know Love,spotify:track:6Aun6tv8JhDBvRwCrXqsK2,Olly Murs:You Don't Know Love,6Aun6tv8JhDBvRwCrXqsK2
2,Hillsong Worship,There Is More,Who You Say I Am - Live/Acoustic,spotify:track:51Q6EroQAuMerQf0Nvfrdh,Hillsong Worship:Who You Say I Am - Live/Acoustic,51Q6EroQAuMerQf0Nvfrdh
3,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,Before You Go,spotify:track:2gMXnyrvIjhVBUZwvLZDMP,Lewis Capaldi:Before You Go,2gMXnyrvIjhVBUZwvLZDMP
4,James Morgan,Veni Creator Spiritus,Veni Creator Spiritus - Chill Mix,spotify:track:0fRCHC5G0vZVadw81H6UKI,James Morgan:Veni Creator Spiritus - Chill Mix,0fRCHC5G0vZVadw81H6UKI


In [82]:
# create final dict as a copy df_stream
df_tableau = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_tableau.head()


,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,2020-09-13 11:36,Burna Boy,Level Up (Twice As Tall) [feat. Youssou N'Dour],80822,Burna Boy:Level Up (Twice As Tall) [feat. Yous...,0,NaN,NaN
1,2020-09-30 11:07,Burna Boy,Level Up (Twice As Tall) [feat. Youssou N'Dour],2990,Burna Boy:Level Up (Twice As Tall) [feat. Yous...,0,NaN,NaN
2,2020-09-30 11:10,Burna Boy,Alarm Clock,132626,Burna Boy:Alarm Clock,0,NaN,NaN
3,2020-09-30 11:13,Burna Boy,Way Too Big,182743,Burna Boy:Way Too Big,1,Twice As Tall,4rlaH7EbXL55bj9pvp57nP
4,2020-09-30 11:16,Tatiana Manaois,Got You Upset,0,Tatiana Manaois:Got You Upset,1,Love Diaries of an Introvert,0dO50htoJSIZQRhXfG8O2q


In [90]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = '8643d70783d04f0e98fef84c3b1931c8'
CLIENT_SECRET = '49246d53829c4227911d801b58694c2e'

In [91]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [92]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [93]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [94]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

In [95]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,7yATeGiKEOcljVe1yet9zx,3whuHq0yGx60atvA2RCVRW,"[dance pop, europop, neo mellow, pop, pop danc..."
1,6Aun6tv8JhDBvRwCrXqsK2,3whuHq0yGx60atvA2RCVRW,"[dance pop, europop, neo mellow, pop, pop danc..."
2,51Q6EroQAuMerQf0Nvfrdh,3SgHzT552wy2W8pNLaLk24,"[ccm, russian ccm, world worship, worship]"
3,2gMXnyrvIjhVBUZwvLZDMP,4GNC7GD6oZMSxPGyXy4MNB,"[pop, uk pop]"
4,0fRCHC5G0vZVadw81H6UKI,7mC0zQ4J4cJPNGYFlmiQVC,[]


In [96]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()



,track_uri,artist_uri,genres
0,7yATeGiKEOcljVe1yet9zx,3whuHq0yGx60atvA2RCVRW,dance pop
0,7yATeGiKEOcljVe1yet9zx,3whuHq0yGx60atvA2RCVRW,europop
0,7yATeGiKEOcljVe1yet9zx,3whuHq0yGx60atvA2RCVRW,neo mellow
0,7yATeGiKEOcljVe1yet9zx,3whuHq0yGx60atvA2RCVRW,pop
0,7yATeGiKEOcljVe1yet9zx,3whuHq0yGx60atvA2RCVRW,pop dance


In [97]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

done
